# Running BOA Optimization Directly in Python

This notebook demonstrates how to:

Write a basic Wrapper in Python and launch a optimization from Python.
If you wanted to launch it from command line, you would do a similar thing of defining the Wrapper, and then put in your configuration file the information about where the wrapper is, and use [BOA's](../index.rst) CLI tools. See [Running an Experiment from Command Line (Python Wrapper)](example_py_run.rst) for more information.

In [1]:
import pathlib
import shutil

from IPython.display import Code
from wrapper import Wrapper

import boa

[WARNING 07-13 14:30:51] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.


In [2]:
# Remove old runs to have a clean slate for this example
old_runs = pathlib.Path().resolve().glob("boa_runs*")
for path in old_runs:
    shutil.rmtree(path)

## Loading the Config File

In [3]:
config_path = pathlib.Path().resolve() / "single_config.yaml"

Here we can see what the configuration file looks like

In [4]:
Code(config_path)

# Single objective optimization config
optimization_options:
    objective_options:
        objectives:
            - name: Cosine8
    trials: 50
    append_timestamp: False
parameters:
    x0:
        type: range
        bounds: [0.0, 1.0]
    x1:
        type: range
        bounds: [0.0, 1.0]
    x2:
        type: range
        bounds: [0.0, 1.0]
    x3:
        type: range
        bounds: [0.0, 1.0]
    x4:
        type: range
        bounds: [0.0, 1.0]
    x5:
        type: range
        bounds: [0.0, 1.0]
    x6:
        type: range
        bounds: [0.0, 1.0]
    x7:
        type: range
        bounds: [0.0, 1.0]
# These are all defaults, so we don't need to specify them in this case
#script_options:
#    wrapper_path: ./wrapper.py
#    wrapper_name: Wrapper
#    working_dir: .
#    experiment_dir: ... # this is where boa will write logs to by default
                         # if not specified it will be working_dir/experiment_name
#    append_timestamp: True
# This last option appends a timestamp to our output experiment directory.
# This is also the default (True)

we need the config normalized, which modifies the parameter section
into a less user friendly form, but what the downstream libraries need

In [5]:
config = boa.load_jsonlike(config_path)

## Define Our Wrapper

We define our wrapper in wrapper.py and use a synthetic function that stands in for any black box model call

In [6]:
Code(Wrapper.path())

import numpy as np
from ax.utils.measurement.synthetic_functions import from_botorch
from botorch.test_functions.synthetic import Cosine8

import boa

cosine8 = from_botorch(Cosine8())


def black_box_model(X) -> float:
    result = -cosine8(X)
    return result


class Wrapper(boa.BaseWrapper):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.data = {}

    def run_model(self, trial) -> None:
        X = np.array([parameter for parameter in trial.arm.parameters.values()])
        # This is a silly toy function, in reality,
        # you could instead import your model main() function and use that, and then collect the results.
        # You could also call an external script to start a model run from Bash or elsewhere.
        self.data[trial.index] = black_box_model(X)

    def set_trial_status(self, trial) -> None:
        data_exists = self.data.get(trial.index)
        if data_exists:
            trial.mark_completed()

    def fetch_trial_data(self, trial, *args, **kwargs):
        return self.data[trial.index]

## Initialize our Setup

In [7]:
controller = boa.Controller(config_path=config_path, wrapper=Wrapper)

controller.initialize_scheduler()

[INFO 07-13 14:30:53] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x0. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-13 14:30:53] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x1. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-13 14:30:53] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-13 14:30:53] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x3. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-13 

(Scheduler(experiment=Experiment(boa_runs), generation_strategy=GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 16 trials, GPEI for subsequent trials]), options=SchedulerOptions(max_pending_trials=10, trial_type=<TrialType.TRIAL: 0>, batch_size=None, total_trials=None, tolerated_trial_failure_rate=0.5, min_failed_trials_for_failure_rate_check=5, log_filepath=None, logging_level=20, ttl_seconds_for_trials=None, init_seconds_between_polls=1, min_seconds_before_poll=1.0, seconds_between_polls_backoff_factor=1.5, timeout_hours=None, run_trials_in_batches=False, debug_log_run_metadata=False, early_stopping_strategy=None, global_stopping_strategy=None, suppress_storage_errors_after_retries=False)),
 <wrapper.Wrapper at 0x16de09780>)

## Run our Experiment

The Controller will save our scheduler to JSON after it completes the run so we can reload it at a later time for analysis or to resume our experiment

In [8]:
scheduler = controller.run()

[INFO 2023-07-13 14:30:53,587 MainProcess] boa: 

##############################################


BOA Experiment Run
Output Experiment Dir: /Users/madelinescyphers/Documents/projs_.nosync/boa/docs/examples/boa_runs_20230713T143053
Start Time: 20230713T143053
Version: 0.8.7.dev4+gae30cf2.d20230713

##############################################

[INFO 07-13 14:30:53] Scheduler: Running trials [0]...
[INFO 07-13 14:30:54] Scheduler: Running trials [1]...
[INFO 07-13 14:30:55] Scheduler: Running trials [2]...
[INFO 07-13 14:30:56] Scheduler: Running trials [3]...
[INFO 07-13 14:30:57] Scheduler: Running trials [4]...
[INFO 07-13 14:30:58] Scheduler: Running trials [5]...
[INFO 07-13 14:30:59] Scheduler: Running trials [6]...
[INFO 07-13 14:31:00] Scheduler: Running trials [7]...
[INFO 07-13 14:31:01] Scheduler: Running trials [8]...
[INFO 07-13 14:31:01] Scheduler: Running trials [9]...
[INFO 07-13 14:31:02] Scheduler: Retrieved COMPLETED trials: 0 - 9.
[INFO 07-13 14:31:02] Scheduler: F

## Get the Best Trial and Output All Trials

`best_fitted_trials` uses the data to do a fitting from all trials and with the noise levels you provided (or if no noise levels was provided, it assumed an unknown level of noise and inferred the noise level from the trial runs)

In [9]:
trial = scheduler.best_fitted_trials()
trial

{49: {'params': {'x0': 0.0,
   'x1': 0.3589494371881439,
   'x2': 0.3635867439663141,
   'x3': 0.3429694617951464,
   'x4': 0.35730864639376125,
   'x5': 0.0,
   'x6': 0.376619314508331,
   'x7': 0.0},
  'means': {'Cosine8': -0.04991830073003367},
  'cov_matrix': {'Cosine8': {'Cosine8': 1.4031232633680942e-06}}}}

if you need the exact points of the best trial, maybe because you need the trial number of the best trial to plot results, or for any other reason, `best_raw_trails` does not do any fitting

In [10]:
trial = scheduler.best_raw_trials()
trial

{47: {'params': {'x0': 0.0,
   'x1': 0.3589494371881439,
   'x2': 0.3635867439663141,
   'x3': 0.3429694617951464,
   'x4': 0.35730864639376125,
   'x5': 0.0,
   'x6': 0.376619314508331,
   'x7': 0.0},
  'means': {'Cosine8': -0.04998773022620064},
  'cov_matrix': {'Cosine8': {'Cosine8': 0.0}}}}

## Output a DataFrame of All Trials

In [11]:
boa.scheduler_to_df(scheduler)

,trial_index,arm_name,trial_status,generation_method,Cosine8,x0,x1,x2,x3,x4,x5,x6,x7
0,0,0_0,COMPLETED,Sobol,2.239105,0.005910,0.141367,0.425610,0.810076,0.635368,0.737481,0.202842,0.678401
1,1,1_0,COMPLETED,Sobol,4.136699,0.341347,0.802491,0.530528,0.982942,0.558278,0.774273,0.698905,0.884335
2,2,2_0,COMPLETED,Sobol,1.838099,0.302048,0.279949,0.768200,0.617394,0.314809,0.049353,0.800600,0.165349
3,3,3_0,COMPLETED,Sobol,2.408117,0.378163,0.272300,0.601005,0.585589,0.075222,0.125974,0.767954,0.901783
4,4,4_0,COMPLETED,Sobol,2.030690,0.283629,0.755290,0.824802,0.287725,0.288463,0.035348,0.849888,0.135928
5,5,5_0,COMPLETED,Sobol,2.714052,0.096529,0.478756,0.827078,0.874856,0.580245,0.656032,0.030549,0.542257
6,6,6_0,COMPLETED,Sobol,3.941725,0.487296,0.817129,0.716379,0.836942,0.999895,0.541045,0.672588,0.288226
7,7,7_0,COMPLETED,Sobol,3.478950,0.582199,0.793198,0.758041,0.570732,0.532217,0.236029,0.958385,0.249541
8,8,8_0,COMPLETED,Sobol,3.067485,0.395262,0.826062,0.535096,0.934967,0.645789,0.598909,0.461659,0.370842
9,9,9_0,COMPLETED,Sobol,3.494017,0.182291,0.651113,0.095934,0.183234,0.885109,0.892330,0.478828,0.958136
